In [1]:
# conda install -c conda-forge gdal
# conda install -c conda-forge geopandas
# conda install -c conda-forge earthpy
# conda install -c conda-forge cloudpathlib
# conda install -c conda-forge pyhdf
# conda install -c anaconda basemap

#conda install -c conda-forge xarray
#conda install -c conda-forge ipywidgets
#conda install -c conda-forge cartopy
## For IO dependencies in xarray 
#conda install -c conda-forge xarray dask netCDF4 bottleneck
#conda install -c conda-forge cfgrib
#conda install -c conda-forge pygrib
#conda install -c yt87 pywgrib2_xr

In [1]:
!jupyter kernelspec list

Available kernels:
  python3    /home/ec2-user/.conda/envs/gfs/share/jupyter/kernels/python3


In [198]:
#Import Packages. 
import sys
import os
import requests
import warnings
import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy.ma as ma
import numpy as np
#from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from osgeo import gdal
import pandas as pd

#GFS data
#import pywgrib2_xr as pywgrib2
import xarray
import pygrib
import xarray # used for reading the data.
import ipywidgets as widgets
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

# #from cloudpathlib import S3Path, S3Client
# from pyhdf.SD import SD, SDC

warnings.simplefilter('ignore')

### [DNU] Using pywgrib2_xr to open grib2 files

In [3]:
!cd ..

In [4]:
!pwd

/local/capstone


In [5]:
in_file = 'gfs.0p25.2015011500.f000.grib2'
out_file = '/tmp/subset.grib2'

match_str = ':(TMP:2 m above ground|[U|V]GRD:10 m above ground):'
pywgrib2.wgrib(in_file, '-inv', '/dev/null', '-match', match_str, '-grib', out_file)
pywgrib2.free_files(in_file, out_file)

In [6]:
with pywgrib2.MemoryBuffer() as buf:
    pywgrib2.wgrib(out_file, '-inv', buf)
    buf.get('s')

In [8]:
files = sorted(glob.glob('gfs.0p25.2015011500.f000.grib2'))
tmpl = pywgrib2.make_template(files, lambda x: x.varname == 'APCP')
ds = pywgrib2.open_dataset(files, tmpl)
ds

AttributeError: 'NoneType' object has no attribute 'item_match'

### [DNU] Downloading GFS Data (0.25 degree)using rdams client

https://github.com/NCAR/rda-apps-clients/blob/main/src/python/rdams_client_example.ipynb

## [Use] Current working method. 

In [4]:
 ## First, we need to authenticate
try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass

In [5]:
## Now, we need your password.
pswd = input('password: ')

password:  ···········


In [6]:
values = {'email' : 'jericojohns@berkeley.edu', 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

In [7]:
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [99]:
dspath = 'https://rda.ucar.edu/data/ds084.1/'
filelist = ['2018/20180101/gfs.0p25.2018010100.f006.grib2']

In [100]:
save_dir = '/local/train/GFS/'

 #### Now to download the files

In [101]:
for file in filelist:
    filename = dspath + file
    outfile = save_dir + os.path.basename(filename)
    print('Downloading', file)
    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
    open(outfile, 'wb').write(req.content)

#### Once you have downloaded the data, the next part can help you plot it.

In order to plot this data, you may need to install librariesIn order to plot this data, you may need to install libraries. The easiest way to do this is to use conda, however any method of getting the following libraries will work.

In [23]:
import xarray # used for reading the data.
import xarray_extras.csv
import ipywidgets as widgets
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

In [102]:
filelist_arr = [save_dir + os.path.basename(file) for file in filelist]
selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
display(selected_file)

Dropdown(description='data file', options=('/local/train/GFS/gfs.0p25.2018010100.f006.grib2',), value='/local/…

In [206]:
# Now to load in the data to xarray
type_of_level = 'surface'
ds = xarray.open_dataset(selected_file.value, filter_by_keys={'typeOfLevel': type_of_level}, engine="cfgrib")

Ignoring index file '/local/train/GFS/gfs.0p25.2018010100.f006.grib2.923a8.idx' incompatible with GRIB file


In [207]:
ds

<xarray.Dataset>
Dimensions:     (latitude: 721, longitude: 1440)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] ...
Data variables: (12/42)
    vis         (latitude, longitude) float32 ...
    gust        (latitude, longitude) float32 ...
    hindex      (latitude, longitude) float32 ...
    sp          (latitude, longitude) float32 ...
    orog        (latitude, longitude) float32 ...
    t           (latitude, longitude) float32 ...
    ...          ...
    4lftx       (latitude, longitude) float32 ...
    hpbl        (latitude, longitude) float32 ...
    lsm         (latitude, longitude) float32 ...
    siconc      (latitude, longitude) float32 ...
    al          (latitude, longitude) float32 ...
    landn       (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2022-02-15T02:26 GRIB to CDM+CF via cfgrib-0.9.1...

In [186]:
ds.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:     (latitude: 721, longitude: 1440)
Coordinates:
    time        datetime64[ns] 2018-01-01
    step        timedelta64[ns] 06:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] 2018-01-01T06:00:00
Data variables: (12/42)
    vis         (latitude, longitude) float32 1.39e+04 1.39e+04 ... 2.41e+04
    gust        (latitude, longitude) float32 2.702 2.702 2.702 ... 10.4 10.4
    hindex      (latitude, longitude) float32 nan nan nan nan ... nan nan nan
    sp          (latitude, longitude) float32 1.007e+05 1.007e+05 ... 6.948e+04
    orog        (latitude, longitude) float32 -0.02375 -0.02375 ... 2.774e+03
    t           (latitude, longitude) float32 246.1 246.1 246.1 ... 251.3 251.3
    ...          ...
    4lftx       (latitude, longitude) float32 14.25 14.25

In [199]:
#Extract valid time (time forecast is in effect. + 6 hours for 6 hour forecast).
#Instead of appending as a row in final csv we will name file using timeestamp. 
valid_time = ds.coords['valid_time'].values
valid_time_str = np.datetime_as_string(valid_time) 
valid_time_str

'2018-01-01T06:00:00.000000000'

In [165]:
ds.variables['t'].values

array([[246.13913, 246.13913, 246.13913, ..., 246.13913, 246.13913,
        246.13913],
       [246.43913, 246.43913, 246.43913, ..., 246.43913, 246.43913,
        246.43913],
       [246.43913, 246.43913, 246.43913, ..., 246.43913, 246.43913,
        246.43913],
       ...,
       [251.23914, 251.23914, 251.23914, ..., 251.33914, 251.33914,
        251.23914],
       [251.33914, 251.33914, 251.33914, ..., 251.33914, 251.33914,
        251.33914],
       [251.33914, 251.33914, 251.33914, ..., 251.33914, 251.33914,
        251.33914]], dtype=float32)

In [208]:
var_name = 't'
ds_filt = ds[var_name]

In [191]:
ds_filt

<xarray.DataArray 't' (latitude: 721, longitude: 1440)>
array([[246.13913, 246.13913, 246.13913, ..., 246.13913, 246.13913, 246.13913],
       [246.43913, 246.43913, 246.43913, ..., 246.43913, 246.43913, 246.43913],
       [246.43913, 246.43913, 246.43913, ..., 246.43913, 246.43913, 246.43913],
       ...,
       [251.23914, 251.23914, 251.23914, ..., 251.33914, 251.33914, 251.23914],
       [251.33914, 251.33914, 251.33914, ..., 251.33914, 251.33914, 251.33914],
       [251.33914, 251.33914, 251.33914, ..., 251.33914, 251.33914, 251.33914]],
      dtype=float32)
Coordinates:
    time        datetime64[ns] 2018-01-01
    step        timedelta64[ns] 06:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] 2018-01-01T06:00:00
Attributes:
    GRIB_paramId:                             130
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  1440
    GRIB_Ny:                                  721
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_gridDefinitionDescription:           Latitude/longitude. Also called...
    GRIB_iDirectionIncrementInDegrees:        0.25
    GRIB_iScansNegatively:                    0
    GRIB_jDirectionIncrementInDegrees:        0.25
    GRIB_jPointsAreConsecutive:               0
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   90.0
    GRIB_latitudeOfLastGridPointInDegrees:    -90.0
    GRIB_longitudeOfFirstGridPointInDegrees:  0.0
    GRIB_longitudeOfLastGridPointInDegrees:   359.75
    GRIB_missingValue:                        9999
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [192]:
ds_filt.shape

(721, 1440)

In [193]:
ds_filt

<xarray.DataArray 't' (latitude: 721, longitude: 1440)>
array([[246.13913, 246.13913, 246.13913, ..., 246.13913, 246.13913, 246.13913],
       [246.43913, 246.43913, 246.43913, ..., 246.43913, 246.43913, 246.43913],
       [246.43913, 246.43913, 246.43913, ..., 246.43913, 246.43913, 246.43913],
       ...,
       [251.23914, 251.23914, 251.23914, ..., 251.33914, 251.33914, 251.23914],
       [251.33914, 251.33914, 251.33914, ..., 251.33914, 251.33914, 251.33914],
       [251.33914, 251.33914, 251.33914, ..., 251.33914, 251.33914, 251.33914]],
      dtype=float32)
Coordinates:
    time        datetime64[ns] 2018-01-01
    step        timedelta64[ns] 06:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  datetime64[ns] 2018-01-01T06:00:00
Attributes:
    GRIB_paramId:                             130
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  1440
    GRIB_Ny:                                  721
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_gridDefinitionDescription:           Latitude/longitude. Also called...
    GRIB_iDirectionIncrementInDegrees:        0.25
    GRIB_iScansNegatively:                    0
    GRIB_jDirectionIncrementInDegrees:        0.25
    GRIB_jPointsAreConsecutive:               0
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   90.0
    GRIB_latitudeOfLastGridPointInDegrees:    -90.0
    GRIB_longitudeOfFirstGridPointInDegrees:  0.0
    GRIB_longitudeOfLastGridPointInDegrees:   359.75
    GRIB_missingValue:                        9999
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [79]:
!pwd

/local/capstone


In [209]:
#Convert to csv, with the appropriate metadata in file name (will extract as field names later). 
filepath = '../train/GFS/' + valid_time_str + '_' + type_of_level + '_' + var_name + '.csv'
xarray_extras.csv.to_csv(x = ds_filt, path = filepath, compression = None)

## Jerico next steps: 
- Confirm t = 'temperature' (email response pending from NCAR w/ full schema)
- Pull in all of Planetary Boundary-Layer Height (PBL), Relative Humidity, Surface Air Temp for 1 file. 
- Before exporting csv: 
    - subset xarray DataArray to relevenat lat/lon bounds 
    - pivot the lat x lon DataArray into a time, (lat,lon), field1, etc. columnar dataset. 
    - Save this columnar, tabular dataset as a csv for a given forecast time and drop the stored grib file from memory. 
    - Iterate to next forecast time grib2 file and repeat until entire date rnage is covered. 
    - Will have ~1095 csv files. These can then be appended into a large DataFrame to join to other datasets (by time, (lat,lon)). 
    

### Downloading MAIAC (MODIS) Data

In [10]:

# s3_cli = S3Client(no_sign_request=True)

# maiac_path = S3Path("s3://drivendata-competition-airathon-public-as/pm25/train/maiac/", client=s3_cli)
# maiac_files = list(maiac_path.rglob("*.hdf"))
# # get first file
# fp = maiac_files[0]
# # download file to temp directory
# fp.fspath
# # download to local
# fp.download_to("1.hdf")

In [15]:
#Open data with gdal
file = gdal.Open('1.hdf')
for path, desc in file.GetSubDatasets():
    print("path:", path, "DESC:", desc)

path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:Optical_Depth_047 DESC: [4x1200x1200] Optical_Depth_047 grid1km (16-bit integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:Optical_Depth_055 DESC: [4x1200x1200] Optical_Depth_055 grid1km (16-bit integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:AOD_Uncertainty DESC: [4x1200x1200] AOD_Uncertainty grid1km (16-bit integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:FineModeFraction DESC: [4x1200x1200] FineModeFraction grid1km (16-bit integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:Column_WV DESC: [4x1200x1200] Column_WV grid1km (16-bit integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:AOD_QA DESC: [4x1200x1200] AOD_QA grid1km (16-bit unsigned integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:AOD_MODEL DESC: [4x1200x1200] AOD_MODEL grid1km (8-bit unsigned integer)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid1km:Injection_Height DESC: [4x1200x1200] Injection_Height grid1km (32-bit floating-point)
path: HDF4_EOS:EOS_GRID:"1.hdf":grid5km:cosSZA DESC: [4x240x240] cosSZ

In [10]:
def load_data(FILEPATH):
    '''Takes the FILEPATH and loads the SUBDATASET_NAME from that file.'''
    ds = gdal.Open(FILEPATH)
    return ds
    
# Opens the data HDF file and returns as a dataframe
def read_dataset(SUBDATASET_NAME, FILEPATH):
    dataset = load_data(FILEPATH)
    path = ''
    for sub, description in dataset.GetSubDatasets():
        if (description.endswith(SUBDATASET_NAME)):
            path = sub
            break
    if(path == ''):
        print(SUBDATASET_NAME + ' not found')
        return
    subdataset = gdal.Open(path)
    subdataset = subdataset.ReadAsArray()
    subdataset = pd.DataFrame(subdataset)
    return subdataset

In [20]:

maiac_fp = "1.hdf" #path to file here
hdf = SD(maiac_fp, SDC.READ)

# construct grid metadata from text blob
gridmeta = hdf.attributes()["StructMetadata.0"]
gridmeta = dict([x.split("=") for x in gridmeta.split() if "=" in x])

for key, val in gridmeta.items():
    try:
        gridmeta[key] = eval(val)
    except:
        pass

In [21]:
gridmeta

{'GROUP': 'PointStructure',
 'END_GROUP': 'PointStructure',
 'GridName': 'grid5km',
 'XDim': 240,
 'YDim': 240,
 'UpperLeftPointMtrs': (12231455.716333, 3335851.559),
 'LowerRightMtrs': (13343406.236, 2223901.039333),
 'Projection': 'GCTP_SNSOID',
 'ProjParams': (6371007.181, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 'SphereCode': -1,
 'GridOrigin': 'HDFE_GD_UL',
 'OBJECT': 'DataField_5',
 'DimensionName': 'Orbits',
 'Size': 4,
 'END_OBJECT': 'DataField_5',
 'DataFieldName': 'Glint_Angle',
 'DataType': 'DFNT_INT16',
 'DimList': ('Orbits', 'YDim', 'XDim')}

In [9]:
import os
import re
import pyproj

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyhdf.SD import SD, SDC
#from mpl_toolkits.basemap import Basemap

FILE_NAME = '1.hdf'
DATAFIELD_NAME = 'Optical_Depth_055'
hdf = SD(FILE_NAME, SDC.READ)

# Read dataset.
data3D = hdf.select(DATAFIELD_NAME)
data = data3D[0,:,:].astype(np.double)

# Read attributes.
attrs = data3D.attributes(full=1)
lna=attrs["long_name"]
long_name = lna[0]
vra=attrs["valid_range"]
valid_range = vra[0]
fva=attrs["_FillValue"]
_FillValue = fva[0]
sfa=attrs["scale_factor"]
scale_factor = sfa[0]        
ua=attrs["unit"]
units = ua[0]
aoa=attrs["add_offset"]
add_offset = aoa[0]

In [13]:
add_offset

0.0

In [10]:
# Apply the attributes to the data.
invalid = np.logical_or(data < valid_range[0], data > valid_range[1])
invalid = np.logical_or(invalid, data == _FillValue)
data[invalid] = np.nan
data = (data - add_offset) * scale_factor
data = np.ma.masked_array(data, np.isnan(data))

# Construct the grid.  The needed information is in a global attribute
# called 'StructMetadata.0'.  Use regular expressions to tease out the
# extents of the grid.
fattrs = hdf.attributes(full=1)
ga = fattrs["StructMetadata.0"]
gridmeta = ga[0]
ul_regex = re.compile(r'''UpperLeftPointMtrs=\(
                          (?P<upper_left_x>[+-]?\d+\.\d+)
                          ,
                          (?P<upper_left_y>[+-]?\d+\.\d+)
                          \)''', re.VERBOSE)

match = ul_regex.search(gridmeta)
x0 = np.float(match.group('upper_left_x'))
y0 = np.float(match.group('upper_left_y'))

lr_regex = re.compile(r'''LowerRightMtrs=\(
                          (?P<lower_right_x>[+-]?\d+\.\d+)
                          ,
                          (?P<lower_right_y>[+-]?\d+\.\d+)
                          \)''', re.VERBOSE)
match = lr_regex.search(gridmeta)
x1 = np.float(match.group('lower_right_x'))
y1 = np.float(match.group('lower_right_y'))
        
nx, ny = data.shape
x = np.linspace(x0, x1, nx, endpoint=False)
y = np.linspace(y0, y1, ny, endpoint=False)
xv, yv = np.meshgrid(x, y)

sinu = pyproj.Proj("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
wgs84 = pyproj.Proj("+init=EPSG:4326") 
lon, lat= pyproj.transform(sinu, wgs84, xv, yv)



/tmp/ipykernel_16943/3154909409.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x0 = np.float(match.group('upper_left_x'))
/tmp/ipykernel_16943/3154909409.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y0 = np.float(match.group('upper_left_y'))
/tmp/ipykernel_16943/3154909409.py:30: DeprecationWarning: `np.float` is a deprecate

In [11]:
lon, lat

(array([[127.01705921, 127.02668171, 127.03630422, ..., 138.53519708,
         138.54481958, 138.55444208],
        [127.00639554, 127.01601724, 127.02563894, ..., 138.52356641,
         138.53318811, 138.54280981],
        [126.99573636, 127.00535725, 127.01497814, ..., 138.51194063,
         138.52156152, 138.53118241],
        ...,
        [117.07815952, 117.08702908, 117.09589863, ..., 127.69501989,
         127.70388945, 127.71275901],
        [117.07195487, 117.08082395, 117.08969304, ..., 127.68825259,
         127.69712168, 127.70599077],
        [117.06575335, 117.07462196, 117.08349058, ..., 127.68148871,
         127.69035732, 127.69922594]]),
 array([[30.        , 30.        , 30.        , ..., 30.        ,
         30.        , 30.        ],
        [29.99166666, 29.99166666, 29.99166666, ..., 29.99166666,
         29.99166666, 29.99166666],
        [29.98333333, 29.98333333, 29.98333333, ..., 29.98333333,
         29.98333333, 29.98333333],
        ...,
        [20.025   

In [14]:
lon.shape

(1200, 1200)